In [6]:
# data generator 
# - using the synthetic images to train our network
import os
import numpy as np
from config import config
from fringes import fringe_wrapper
from fringes import Fringes_Generator 
import random
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter   
import torch,gc

cfg = config()
cfg.net_dir="./data/train3/"
cfg.model_dir = "./data/model3/"
cfg.tensorboard_dir = "./path/to/log/"
cfg.pattern_size = [1920, 50] #[1920, 50] [512, 1]
cfg.Tp = [30, 33, 36]       #在这个实验中，就先设置成这个周期。
cfg.steps = [7,4,3]
cfg.gamma = 1.1

#删除文件
def del_files(path_file):
    ls = os.listdir(path_file)
    for i in ls:
        f_path = os.path.join(path_file, i)
        # 判断是否是一个目录,若是,则递归删除
        if os.path.isdir(f_path):
            del_files(f_path)
        else:
            os.remove(f_path)

a = input("是否需要重新生成数据集，请输入Y或者N ：")
if a == 'Y':
    if not os.path.exists(cfg.net_dir):
        os.makedirs(cfg.net_dir)
    del_files(cfg.net_dir)
    # #gamma
    # for i in range(100):
    #     images = fringe_wrapper(cfg, "gamma").generate_all()
    #     np.savez(f"./data/train3/train_data{i:03d}", image1=images[0], image2=images[1], image3=images[2]) 

    # #harmonic
    for ind in range(100):
        print(ind)
        fringe_generator=Fringes_Generator(cfg)
        fringe_generator.save_data(ind)



b = input("是否需要重新开始训练，请输入Y或者N ：")
if b == 'Y':
    if not os.path.exists(cfg.model_dir):
        os.makedirs(cfg.model_dir)
    del_files(cfg.model_dir)


In [7]:
# build a MLP model for residual estimation 
#- The linear part is also considered in this network
from torch import nn

# 定义训练的设备
device = torch.device("cuda") #使用gpu进行训练
gc.collect()
torch.cuda.empty_cache()#清楚cuda缓存

class NeuralNetwork(nn.Module): #构建CNN神经网络
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.MLP = nn.Sequential(
            nn.Conv2d(9,  64, 1, stride=1),  #四个卷积层
            nn.ReLU(),
            nn.Conv2d(64, 256, 1, stride=1),
            nn.ReLU(),
            nn.Conv2d(256, 128, 1, stride=1),
            nn.ReLU(),
            nn.Conv2d(128, 4, 1, stride=1),
        )
        
        A = [[1, np.cos(0), np.sin(0)],                
             [1, np.cos(6*np.pi/7), np.sin(6*np.pi/7)],
             [1, np.cos(8*np.pi/7), np.sin(8*np.pi/7)]]
        B = [[1, np.cos(2*np.pi/7), np.sin(2*np.pi/7)],
             [1, np.cos(4*np.pi/7), np.sin(4*np.pi/7)],
             [1, np.cos(10*np.pi/7), np.sin(10*np.pi/7)],
             [1, np.cos(12*np.pi/7), np.sin(12*np.pi/7)]]
        A, B = np.array(A), np.array(B) #列表转化成数组
        self.C = torch.Tensor(np.matmul(B,np.linalg.inv(A)).transpose()).float()#B乘以A的转置矩阵就是I4到I7的前面的系数
        self.C = self.C.to(device)
        print("self.C : ",self.C)
        D = [[1, np.cos(0), np.sin(0)],
            [1, np.cos(2*np.pi/4), np.sin(2*np.pi/4)],
            [1, np.cos(4*np.pi/4), np.sin(4*np.pi/4)]]
        E = [[1, np.cos(6*np.pi/4), np.sin(6*np.pi/4)]]
        self.F = torch.Tensor(np.matmul(E, np.linalg.pinv(D)).transpose()).float()#B乘以A的转置矩阵就是I4到I7的前面的系数
        self.F = self.F.to(device)
        self.model_save_dir = "./data/model3" #定义保存模型的文件夹
        self.p = os.path.join(self.model_save_dir,"best_model")
        self.last_loss = 9999999999
        self.last_epoch = 0
        
        #对数组A求逆矩阵，与B矩阵相乘
    
    def forward(self, images):
        # linear generator for the 4 images
        images = images.to(device)
        i7s = images.permute(0, 2, 3,1)            #将tensor的维度换位
        # i7s = i7s.to(device)
        i7s,_ = torch.split(i7s,(3,6),dim=-1)      #切分，得到的就是I1 I7 I8
        # mids,another = torch.split(_,(3,3),dim = -1) #切分，得到4步相移的前三步
        # print("self.C.shape",self.C.shape)
        # print("i7s.shape",i7s.shape)

        linearPart = torch.matmul(i7s, self.C+0.0) #Y矩阵乘法 50*(1950*3)矩阵乘以3*4矩阵
        # linearPart1 = torch.matmul(mids, self.F+0.0)   #矩阵乘法 50*(1950*3)矩阵乘以3*1矩阵
        linearPart = linearPart.to(device)
        # linearPart1 = linearPart1.to(device)
        linearPart = linearPart.permute(0,3,1,2)   #维度换位
        # linearPart1 = linearPart1.permute(0,3,1,2)   #维度换位
        # linearPart2 = torch.cat([linearPart, linearPart1], 1)
        
        # The nonliear part is estimated with MLP function 
        images = images.to(device)
        images = images/255
        images = images.to(device)
        res = self.MLP(images)
        return linearPart + res
    
    def save(self,optimizer,epoch,loss,batch): #TODO
        if not os.path.exists(self.model_save_dir):
            os.makedirs(self.model_save_dir)
        m = os.path.join(self.model_save_dir,"model"+str(epoch))

        state = {'model':self.state_dict(), 'optimizer':optimizer.state_dict(), 'epoch':epoch}
        # 保存模型
#         if epoch % 100 == 0:
#             torch.save(state, m)
#             print("模型{}保存成功".format(str(epoch)))
        
        if (epoch == 0 or self.last_loss == 10000):
            self.last_loss = loss
            self.last_epoch = epoch
            torch.save(state, self.p)
            # print("lat_loss {},best_model保存成功,它是epoch{},loss为{},batch为{}".format(self.last_loss,epoch,loss,batch))
        else:
            if (loss <= self.last_loss): #用loss判断最优模型
                self.last_loss = loss
                self.last_epoch = epoch
                torch.save(state, self.p)
                # print("lat_loss {},best_model保存成功,它是epoch{},loss为{},batch为{}".format(self.last_loss,epoch,loss,batch))
            else:
                pass
                # print("best_model未保存，最优模型仍为epoch{}".format(self.last_epoch,self.last_loss))
        
    def load(self,optimizer): #TODO

        # 如果有保存的模型，则加载模型，并在其基础上继续训练
        if os.path.exists(self.p):
            checkpoint = torch.load(self.p)
            self.load_state_dict(checkpoint['model'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            start_epoch = checkpoint['epoch']
            print('加载best_model成功！其为epoch{}'.format(start_epoch))
        else:
            start_epoch = 0
            print('无保存模型，将从头开始训练！')
        return start_epoch

    
PSP_Rnet = NeuralNetwork()
PSP_Rnet = PSP_Rnet.to(device)

learning_rate = 2e-2
batch_size = 5

# Initialize the loss function
loss_fn = nn.MSELoss() #均方误差
loss_fn = loss_fn.to(device)

# optimizer = torch.optim.SGD(PSP_Rnet.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(PSP_Rnet.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

start_epoch = PSP_Rnet.load(optimizer)

# Test the network
# TestTensor=torch.Tensor(1,9,1024,512) #数值从0到1，分为9块，每一块中1024行，512列
# result = PSP_Rnet(TestTensor)
# print(result.shape)

self.C :  tensor([[ 0.8019,  0.3569,  0.3569,  0.8019],
        [ 1.0000,  1.4450, -0.8019, -0.8019],
        [-0.8019, -0.8019,  1.4450,  1.0000]], device='cuda:0')
无保存模型，将从头开始训练！


In [8]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import cv2

class CustomImageDataset(Dataset):#自定义图像数据集
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(os.listdir(self.img_dir))

    def __getitem__(self, idx):
        data_name =f"train_data{idx:03d}.npz"
        data_path = os.path.join(self.img_dir, data_name)
        data = np.load(data_path)
        img1_0, img1_3, img1_4 = data["image1"][0], data["image1"][3], data["image1"][4]
        img2_0, img2_1, img2_2 = data["image2"][0], data["image2"][1], data["image2"][2]
        img3_0, img3_1, img3_2 = data["image3"][0], data["image3"][1], data["image3"][2]
        img1_1, img1_2, img1_5, img1_6, img2_3 = data["image1"][1], data["image1"][2], data["image1"][5], data["image1"][6], data["image2"][3]
        inputs = np.stack([img1_0, img1_3, img1_4, img2_0, img2_1, img2_2, img3_0, img3_1, img3_2]).astype(np.float32)
        outputs = np.stack([img1_1, img1_2, img1_5, img1_6]).astype(np.float32)


        # img11_0, img11_3, img11_4 = data["image4"][0], data["image4"][3], data["image4"][4]
        # img22_0, img22_1, img22_2 = data["image5"][0], data["image5"][1], data["image5"][2]
        # img33_0, img33_1, img33_2 = data["image6"][0], data["image6"][1], data["image6"][2]

        # inputs1 = np.stack([img11_0, img11_3, img11_4, img22_0, img22_1, img22_2, img33_0, img33_1, img33_2]).astype(np.float32)
#         print(inputs.shape)
#         print(outputs.shape)

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return inputs,outputs
    
training_data = CustomImageDataset(cfg.net_dir)
inputs, outputs = training_data[0]
# print(inputs.shape)


In [9]:
train_dataloader = DataLoader(training_data, batch_size)
# test_dataloader = DataLoader(test_data, batch_size=64)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer,loss_list):
    cost = []
    size = len(dataloader.dataset)
    # print("size : ",size)
    for data in enumerate(dataloader):#batch, (X, y)
        batch, imgs = data
        # batch = batch.to(device)
        # imgs = imgs.to(device)
        # Compute prediction and loss
#         print("X.shape : ",X.shape)
# #         print(y)
# #         print(X)
#         print("y.shape : ",y.shape)
        X,y = imgs
        X = X.to(device)
        y = y.to(device)
        # print("batch : ",batch)
        
        model = model.to(device)
        pred = model(X)          # 前向传播计算预测值
        # pred_type = pred.is_cuda
        pred = pred.to(device)
        loss_fn = loss_fn.to(device)
        loss = loss_fn(pred, y)  #真实值与预测值求均方误差
        # print("pred",pred)
        # print("y",y)
#         print(pred-y)
        # Backpropagation
        optimizer.zero_grad()   # 将模型的参数梯度初始化为0
        loss = loss.to(device)
        loss.backward()         # 反向传播计算梯度
        optimizer.step()        # 更新所有参数

#         if batch % 100 == 0:
#             model.save()
        loss, current = loss.item(), batch * len(X)
        cost.append(loss)
        if len(loss_list) < 100//batch_size:
            loss_list.append(loss)
        else:    
            loss_list[batch] = loss
            
        # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        PSP_Rnet.save(optimizer,t,loss,batch)
    print(optimizer.state_dict()['param_groups'][0]['lr'])
    print(f"Epoch loss average:{np.mean(cost)}")
    return np.mean(cost)
        


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

if not os.path.exists(cfg.tensorboard_dir):
    os.makedirs(cfg.tensorboard_dir)
del_files(cfg.tensorboard_dir)
writer = SummaryWriter(cfg.tensorboard_dir)
epochs = 200000
loss_list = []
for t in range(start_epoch,epochs):
    print(f"Epoch {t}\n-------------------------------")
    loss = train_loop(train_dataloader, PSP_Rnet, loss_fn, optimizer,loss_list)
    scheduler.step() # 需要在优化器参数更新之后再动态调整学习率
    
    writer.add_scalar('manner3_gpu_loss',loss, t)
    # writer.add_scalars('manner1_gpu_loss',{'batch0':loss_list[0],'batch1':loss_list[1],'batch2':loss_list[2],'batch3':loss_list[3],'batch4':loss_list[4],'batch5':loss_list[5],'batch6':loss_list[6]}, t)
    sum = 0
    element = 0
    for i in loss_list:
        element += 1
        if i <= 0.2:
            sum += 1
    if sum == element and element > 0:
        break
#     test_loop(test_dataloader, PSP_Rnet, loss_fn)

print("Done!")

Epoch 0
-------------------------------
